In [1]:
import yfinance as yf
import pandas as pd
#import talib as tb
import numpy as np
import ta
import matplotlib.pyplot as plt
import seaborn as sns
from spare_parts import get_company_list


In [ ]:
ta.trend.MACD(data_raw['Close']).macd()

In [ ]:
data_raw  = pd.read_csv("backtesting/tcs_practice_data.csv")

In [ ]:
ta.trend._sma(data_raw['Close'],20)

In [ ]:
# applying [Bollinger Band OBV]
obv = tb.OBV(data_raw['Close'], data_raw['Volume']) #type:ignore

In [ ]:
obv

In [ ]:
data_raw.head(10)

In [ ]:
plt.plot(data_raw['Close'])

In [ ]:
buy_date = None
sell_date = None
current_position_number = 0
MAX_PROFIT_LIMIT = 0.2
MAX_LOSS_LIMIT =  -0.1

In [ ]:
data_raw.columns

In [ ]:
x = [1,2,3,4,5,6,7,8,9,10] # current_count = 8
threshold = 7
current_count = 8
if(current_count <  threshold):
    print("calculation not possible")
else:
    print(x[current_count] - x[current_count-threshold])


In [ ]:
threshold = 7
current_count = 0
actions  = []
logs = []
data_raw_list = data_raw[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].to_numpy()
for i in range(len(data_raw_list)):
    current_price = data_raw_list[i][3]  # closing price
    if(i < threshold):
        continue
    else:
        last_seven_days_price = data_raw_list[i-threshold][3]
        print((current_price-last_seven_days_price)/current_price,"-")
        if(current_price - last_seven_days_price/current_price > 0.1 and current_position_number <=  2):
            current_position_number += 1
            print("bought stock at {}".format(current_price))
            buy_date = data_raw.index[i]
            logs.append(current_price)

        # selling the stock with profit
        if(len(logs) != 0):
          
            max_log_price = max(logs)
            if((current_price - max_log_price) / current_price > 0.2):
                print("sold stock at {} in profit".format(current_price))
                logs.pop(logs.index(max_log_price))
                current_position_number -= 1
                sell_date = data_raw.index[i]
                # appending all the action to main list
                actions.append([sell_date, current_price - max_log_price,"profit"])
        if(len(logs) != 0):
            max_log_price = max(logs)
            if((current_price - max_log_price) / current_price < -0.1):
                print("sold stock at {} in loss".format(current_price))
                logs.pop(logs.index(max_log_price))
                current_position_number -= 1
                sell_date = data_raw.index[i]
                # appending all the action to main list
                actions.append([sell_date, current_price - max_log_price, "loss"])
    

            

        
            
            


           
    
    



In [ ]:
data_raw

In [ ]:
# converting list to dataframe
refined_data =  pd.DataFrame(actions, index = None, columns = ['date', 'profit/loss', 'outcome'])
refined_data.index = refined_data['date']  # type: ignore
refined_data.drop(['date'], axis = 1, inplace = True)

In [ ]:
refined_data['outcome'].value_counts()

In [ ]:
# checking net profit- loss
refined_data['profit/loss'].sum()

<h1> Trend Following Strategies</h1>

In [4]:
""" 
['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ma_50','ma_200', 'ma_50_200']
"""
class ma_trading_strategy:
    def __init__(self, ticker):  # can  use ecternal dataset if provided
        # values can be changed here
        self.data_list = self.download_data(ticker)
        self.MAX_PROFIT_LIMIT = 0.2
        self.ticker = ticker
        self.MAX_LOSS_LIMIT =  -0.2
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []

    def download_data(self, ticker, interval = "1d"):
        data_raw = yf.download(ticker ,period = "max" , progress=True, interval=interval)
        # adding ma_50 and ma_200
        data_raw['ma_50'] = ta.trend._ema(data_raw['Close'], 50)  # type: ignore
        data_raw['ma_200'] = ta.trend._ema(data_raw['Close'], 200) # type: ignore
        data_raw['ma_20'] = ta.trend._ema(data_raw['Close'], 20) # type: ignore

        data_raw.dropna(inplace = True)
        data_raw['Date']  =data_raw.index
        data_raw['ma_50_200'] = data_raw['ma_50'] > data_raw['ma_200']
        data_raw['ma_20_200'] = data_raw['ma_20'] > data_raw['ma_200']

        data_raw = data_raw.reindex(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 
                                               'Volume', 'ma_20','ma_50','ma_200','ma_20_200','ma_50_200'])

        #print(data_raw.columns)
        #print(data_raw.index)
        return  data_raw.to_numpy()
    def print_data(self):
        print(self.data_list)
    # main funciton

    def run_test(self):
        for i in range(len(self.data_list)):
            #print(i)
            try:
                # buying section
                current_node = self.data_list[i]
                previous_node = self.data_list[i-1]
                #print(current_node[11], previous_node[11])
                if(self.current_position <= 2 and [previous_node[11],current_node[11]] == [False, True]):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                
                # selling section
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    # selling the stock with profit
                    if((current_node[4] - max_position) / current_node[4]  >= self.MAX_PROFIT_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # apping all the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "profit"])
                    # selling the stock with loss
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # appending the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
                
            except:
                pass
    
    # extension functions
    def get_output(self):
        return self.output
    
    def get_logs(self):
        return self.logs


            


In [ ]:
ma_trading_strategy("TCS.NS").run_test()

In [ ]:
# this class sells the stock when another [50-200] crossover happens
class ma_trading_strategy_2(ma_trading_strategy):
    def __init__(self, ticker):
        super().__init__(ticker)
        # addintional attributes
        
        self.experiment = 0
    def run_test(self):
        for i in range(len(self.data_list)):
            # buying stock if condition satisfies
            try:
                current_node = self.data_list[i]
                previous_node = self.data_list[i-1]
                if(self.current_position <= 2 and [previous_node[11],current_node[11]] == [False, True]):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                
                # selling stock if condition satisfies
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    # selling the stock with profit
                    if([previous_node[11], current_node[11]] == [True, False]):
                        self.current_position -= 1
                        self.positions.pop(self.positions.index(max_position))
                        self.output += current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], current_node[4] - max_position))
                        self.logs.append([current_node[0],current_node[4], current_node[4] - max_position, "profit"])
                    # selling the stock with loss
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):  # applying - loss limit
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # appending the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            except:
                pass
            
                    
                
                



<h1>  Trend following using Momentum strateies </h1>

In [66]:
# Macd based croosover strategy and exiting after machieving some percentage of profit
class macd_strategy:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT =  -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        self.data_list = self.download_data()
    # basic info  function
    def info(self):
        return self.data_list

    # downloading the data for a given ticker
    def download_data(self):
        data_raw = yf.download(self.ticker+".NS",period="max" ,progress=False, interval="1d")
        # adding ma_50 and ma_200
        data_raw['Date'] = data_raw.index
        alfa = ta.trend.MACD(data_raw['Close'], window_fast=9, window_slow=26) # type: ignore
        macd = alfa.macd()
        macdsignal = alfa.macd_signal()
        data_raw['macd'] = macd
        data_raw['macdsignal'] = macdsignal
        data_raw.dropna(inplace = True)
        data_raw['signal'] = data_raw['macd'] > data_raw['macdsignal']
        data_raw = data_raw.reindex(columns=['Date','Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                                    'macd','macdsignal', 'signal'])
        return data_raw.to_numpy()
    
    # main funciton

    def run_test(self):
        for i in range(len(self.data_list)):
            try:
                current_node = self.data_list[i]
                previous_node = self.data_list[i-1]
                # buying section
                if(self.current_position <= 2 and [previous_node[9],current_node[9]] == [False, True]):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                # selling section
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    # selling the stock with profit
                    if((current_node[4] - max_position) / current_node[4]  >= self.MAX_PROFIT_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # apping all the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "profit"])
                    # selling the stock with loss
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        # appending the events in logs
                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            except:
                pass
        
    
    def get_output(self):
        return self.output
    
            
            





In [22]:
# creating new strategy
class bollinger_band:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT = -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        # special - signal 
        self.signal = 0 # this signal changes once stock reaches the upper - bollinger band
    # define function
    def download_data(self):
        data_raw = yf.download(self.ticker+".NS", period="max", interval = "1d")
        # calculating bollinger band
        volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore
        data_raw['bollinger_hband'] = volatile.bollinger_hband()
        data_raw['bollinger_lband'] = volatile.bollinger_lband()
        data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
        data_raw['bollinger_pband'] = volatile.bollinger_pband()
        # calculating obv
        volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore
        data_raw['obv'] = volatile_obv.on_balance_volume()
        # calculating rsi
        data_raw['rsi'] = ta.momentum.RSIIndicator(data_raw['Close']).rsi() #type: ignore
        data_raw.dropna(inplace = True)
        data_raw['diff'] = data_raw['obv'].diff(periods=10) # slope
        print(data_raw.columns)

        self.data_list = data_raw.to_numpy()
        return data_raw
    # main function
    def run_test(self):
        self.download_data()
        for i in range(len(self.data_list)):
            # --
            try:
                current_node = self.data_list[i]
                # checking the bollinger signal 
                if(self.signal == 0):
                    if(current_node[6] >=  current_node[3]):
                        self.signal = 1
                # buying section
                if(self.current_position <= 2 and self.signal == 1):
                    # checking [RSI] and [OBV - diff] condition
                    if(current_node[12] > 0 and current_node[11] > 50):
                        self.current_position += 1
                        self.positions.append(current_node[3])
                        self.signal = 0
                # selling section

                # selling with profit condition
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    #(current_node[3] - max_position) / current_node[3]  >= self.MAX_PROFIT_LIMIT
                    if(current_node[3] < current_node[7]):
                        net_output = current_node[3] - max_position
                        print("selling the stock at {} with profit/loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                        self.logs.append([current_node[0],current_node[3], net_output, "profit"])
                # selling with loss condititon
            except:
                pass
    
    def get_output(self):
        return self.output


In [23]:
alfa = bollinger_band("RELIANCE")
alfa.run_test()

[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'bollinger_hband', 'bollinger_lband', 'bollinger_mavg',
       'bollinger_pband', 'obv', 'rsi', 'diff'],
      dtype='object')
selling the stock at 15.225051879882812 with profit/loss -3.109996795654297
selling the stock at 15.225051879882812 with profit/loss -2.9939517974853516
selling the stock at 17.65812110900879 with profit/loss 0.47191429138183594
selling the stock at 17.708406448364258 with profit/loss 1.2996997833251953
selling the stock at 17.24422836303711 with profit/loss 0.9128837585449219
selling the stock at 14.923336029052734 with profit/loss -3.5819091796875
selling the stock at 15.147688865661621 with profit/loss -2.7502565383911133
selling the stock at 19.828155517578125 with profit/loss 2.7541255950927734
selling the stock at 20.33101463317871 with profit/loss -0.08123207092285156
selling the stock at 20.19949722290039 with profit/

In [24]:
alfa.get_output()

4693.594668388367

<h1> RSI based strategy </h1>

In [15]:
class rsi_trading():
    def __init__(self, ticker):

        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT =  -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        self.data_list = self.download_data_1()
    

    def download_data_1(self):
        data_raw = yf.download(self.ticker+".NS" , period = "max", interval = "1d")
        data_raw['Date'] = data_raw.index  #type: ignore
        data_raw['rsi'] = ta.momentum.rsi(data_raw['Close']) #type: ignore
        data_raw.dropna(inplace = True)
        print(data_raw.columns)
        self.data_list = data_raw.to_numpy()
        return data_raw.to_numpy()
    
    # getting data lsit info
    
    def info(self):
        return self.data_list
    
    # buying and selling the stocks
    def run_test(self):
        for i in range(len(self.data_list)):
            try:
                current_node = self.data_list[i]
                #previous_node = self.data_list[i-1]
                # buying section
                if(self.current_position <= 2 and current_node[7] < 30):
                    self.current_position += 1
                    self.positions.append(current_node[4])
                
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    if((current_node[4] - max_position) / current_node[4]  >= self.MAX_PROFIT_LIMIT):

                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                    # selling the stock if loss occured
                    if((current_node[4] - max_position) / current_node[4]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[4] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output

                        self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            
            except:
                pass
        
    
    def get_output(self):
        return self.output
        
        

In [ ]:
alfa = rsi_trading("TCS")
alfa.run_test()

In [ ]:
alfa.get_output()

<h1> Playground </h1>

In [4]:
company_list = get_company_list()
len(company_list)

2012

In [33]:
company_list

(('Reliance Industries Limited', 'RELIANCE'),
 ('Tata Consultancy Services Limited', 'TCS'),
 ('HDFC Bank Limited', 'HDFCBANK'),
 ('Infosys Limited', 'INFY'),
 ('ICICI Bank Limited', 'ICICIBANK'),
 ('Hindustan Unilever Limited', 'HINDUNILVR'),
 ('State Bank of India', 'SBIN'),
 ('Bajaj Finance Limited', 'BAJFINANCE'),
 ('Housing Development Finance Corporation Limited', 'HDFC'),
 ('Bharti Airtel Limited', 'BHARTIARTL'),
 ('Kotak Mahindra Bank Limited', 'KOTAKBANK'),
 ('Wipro Limited', 'WIPRO'),
 ('HCL Technologies Limited', 'HCLTECH'),
 ('ITC Limited', 'ITC'),
 ('Adani Green Energy Limited', 'ADANIGREEN'),
 ('Asian Paints Limited', 'ASIANPAINT'),
 ('Bajaj Finserv Limited', 'BAJAJFINSV'),
 ('Adani Transmission Limited', 'ADANITRANS'),
 ('Avenue Supermarts Limited', 'DMART'),
 ('Larsen & Toubro Limited', 'LT'),
 ('Adani Total Gas Limited', 'ATGL'),
 ('Axis Bank Limited', 'AXISBANK'),
 ('Maruti Suzuki India Limited', 'MARUTI'),
 ('Titan Company Limited', 'TITAN'),
 ('Adani Enterprises Lim

In [34]:
net_output = {}
for i in company_list[300:600]:
    try:
    
        rsi_strategy = bollinger_band(i[1])
        rsi_strategy.run_test()
        net_output[i[1]] = rsi_strategy.get_output()
        
    except:
        pass
   
    

[*********************100%***********************]  1 of 1 completed
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'bollinger_hband', 'bollinger_lband', 'bollinger_mavg',
       'bollinger_pband', 'obv', 'rsi', 'diff'],
      dtype='object')
selling the stock at 5.853030204772949 with profit/loss 0.3070230484008789
selling the stock at 5.585200786590576 with profit/loss 0.11105060577392578
selling the stock at 5.617863178253174 with profit/loss 0.22863435745239258
selling the stock at 8.315744400024414 with profit/loss 2.443117618560791
selling the stock at 9.29560375213623 with profit/loss 0.5552539825439453
selling the stock at 9.243345260620117 with profit/loss 3.416445255279541
selling the stock at 9.093099594116211 with profit/loss 3.279264450073242
selling the stock at 12.202522277832031 with profit/loss 2.52803897857666
selling the stock at 12.104536056518555 with profit/loss 2.4561824798583984
selling the stock at 11.954291343688965 with profit/loss 2.345

In [35]:
net_output

{'CHOLAHLDNG': 1462.8079266548157,
 'TTKPRESTIG': 1207.0358206629753,
 'KEI': 3439.0098724365234,
 'REDINGTON': 420.56004428863525,
 'CAMS': 1110.94921875,
 'MAHABANK': 98.99999046325684,
 'KIMS': 198.2998046875,
 'HINDCOPPER': -1519.3499374389648,
 'IIFL': 0,
 'HFCL': 15.749995231628418,
 'SHRIRAMCIT': 0,
 'CGCL': 1576.1154985427856,
 'SJVN': -10.049993515014648,
 'VIPIND': 2016.6597938537598,
 'LXCHEM': 401.05003356933594,
 'CHEMPLASTS': -723.2498779296875,
 'ASAHIINDIA': 1277.3440467119217,
 'METROPOLIS': 3915.7993774414062,
 'LAXMIMACH': 36210.786083221436,
 'CYIENT': 1334.4751205444336,
 'JSL': 948.1500101089478,
 'GALAXYSURF': 3626.1998291015625,
 'BLUESTARCO': 3819.2301301956177,
 'BDL': 1076.1500396728516,
 'CESC': 145.6529220342636,
 'BALRAMCHIN': 1444.4020318984985,
 'CASTROLIND': 133.73126029968262,
 'MASTEK': 2515.6046295166016,
 'RHIM': 1896.7499961853027,
 'ASTERDM': -38.04998016357422,
 'IDFC': 76.23386478424072,
 'GRAPHITE': 2446.8399806022644,
 'QUESS': 484.20022583007

In [7]:
net_output.keys()

dict_keys(['RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'ICICIBANK', 'HINDUNILVR', 'SBIN', 'BAJFINANCE', 'HDFC', 'BHARTIARTL', 'KOTAKBANK', 'WIPRO', 'HCLTECH', 'ITC', 'ADANIGREEN', 'ASIANPAINT', 'BAJAJFINSV', 'ADANITRANS', 'DMART', 'LT', 'ATGL', 'AXISBANK', 'MARUTI', 'TITAN', 'ADANIENT', 'SUNPHARMA', 'ONGC', 'ULTRACEMCO', 'JSWSTEEL', 'ADANIPORTS', 'TATASTEEL', 'POWERGRID', 'VEDL', 'TECHM', 'TATAMOTORS', 'HINDZINC', 'NTPC', 'HINDALCO', 'PIDILITIND', 'DIVISLAB', 'HDFCLIFE', 'COALINDIA', 'SBILIFE', 'IOC', 'GRASIM', 'LTI', 'BAJAJ-AUTO', 'M&M', 'DABUR', 'DLF', 'SHREECEM', 'SIEMENS', 'CIPLA', 'SBICARD', 'NYKAA', 'SRF', 'BPCL', 'INDIGO', 'BRITANNIA', 'GODREJCP', 'TATAPOWER', 'INDUSINDBK', 'HAVELLS', 'ICICIPRULI', 'TATACONSUM', 'DRREDDY', 'ADANIPOWER', 'MINDTREE', 'GAIL', 'BERGEPAINT', 'EICHERMOT', 'AWL', 'ICICIGI', 'MARICO', 'APOLLOHOSP', 'ZOMATO', 'MCDOWELL-N', 'MPHASIS', 'IRCTC', 'INDUSTOWER', 'AMBUJACEM', 'CHOLAFIN', 'UPL', 'NAUKRI', 'BANKBARODA', 'BAJAJHLDNG', 'TATAELXSI', 'JINDALSTEL', 'LODHA'

In [39]:
data_1 = pd.DataFrame()
data_1['company'] = list(net_output.keys())
data_1['net_output'] = list(net_output.values())
data_1['net_output'].sum()

432620.52552197874

In [38]:
data_1[data_1['net_output'] < 0].count()

company       58
net_output    58
dtype: int64

In [20]:
data_1.to_csv("backtesting/bollinger_band_2.csv")

In [13]:
data_raw = pd.read_csv("backtesting/macd_strategy.csv")


,Unnamed: 0,company,net_output
0,0,RELIANCE,5502.143812
1,1,TCS,4831.794735
2,2,HDFCBANK,2705.980113
3,3,INFY,1979.965470
4,4,ICICIBANK,2185.927317
...,...,...,...
263,263,ZFCVINDIA,10026.899811
264,264,UCOBANK,-109.949987
265,265,NUVOCO,-381.800018
266,266,NATCOPHARM,525.050012


In [14]:
data_raw['net_output'].sum()

1009212.2995135039

In [12]:
data_1[data_1['net_output'] < 0].count()

company       49
net_output    49
dtype: int64

In [ ]:
sns.heatmap(data_1.corr(), annot=True)

In [ ]:
alfa = data_1.corr(method="pearson")


In [ ]:
alfa

In [ ]:
alfa[alfa > .90].fillna("gama", inplace = True)

In [ ]:
alfa.index

In [ ]:
alfa.columns

In [ ]:
alfa.to_numpy()

In [ ]:
# getting companies of custom correlation pair

""" 
pair- selection rate -> .70
"""
data_1 = list(alfa.columns)
data_2 = list(alfa.index)
pairs = []
count = -1
for i in alfa.to_numpy():
    count += 1
    
    for i1 in range(len(list(i))):
        # print(i[i1])

         
         if(i[i1] >= .70 and i[i1] != 1.0):
                #sprint(i[i1])
                try:
                    pairs.append((data_1[i1],data_2[count],i[i1]))
                     
                except:
                     
                    pass
     
pairs

In [ ]:
# creating dataframe out of refined data
pair_trading = pd.DataFrame(pairs, columns=['companies_1', 'companies_2', 'correlation'])
pair_trading.drop_duplicates(inplace = True)

In [ ]:
icicibank_data= yf.download("ICICIBANK.NS", period= "1y", interval = "1d")
hdfcbank_data = yf.download("HDFCBANK.NS", period="1y", interval = "1d")
plt.figure(figsize = (20, 10))
plt.plot(icicibank_data['Close'])
plt.plot(hdfcbank_data['Close'])

In [ ]:
icicibank_data.to_csv("icic.csv")
hdfcbank_data.to_csv("hdfc.csv")

In [ ]:
# plotting the data
new_data = yf.download("SBIN.NS", period ="5y" , interval="1d")
plt.figure(figsize=(20,10))
plt.plot(new_data['Close'])
plt.plot(tb.MA(new_data['Close'], timeperiod=100)) # type: ignore

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(new_data['Close'])
plt.plot(tb.MA(new_data['Close'], timeperiod=50)) # type: ignore

In [ ]:
# applying visulization and anaysis
new_dataframe = pd.DataFrame(net_output, columns = ['company', 'net_output'])
#new_dataframe.to_csv("simple_ma_strategy.csv")

In [ ]:
new_dataframe['net_output'].sum()

In [ ]:
new_dataframe[new_dataframe['net_output'] < 0].count()

<h1> [RSI + OBV + MACD] </h1>

In [ ]:
# generating bollinger bank and obv
data_raw = pd.read_csv("backtesting/tcs_practice_data.csv")
volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore

In [ ]:
volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore


In [ ]:
data_raw['obv'] = volatile_obv.on_balance_volume()

In [ ]:
data_raw['bollinger_hband'] = volatile.bollinger_hband()
data_raw['bollinger_lband'] = volatile.bollinger_lband()
data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
data_raw['bollinger_pband'] = volatile.bollinger_pband()

In [ ]:
data_raw.dropna(inplace = True)

In [ ]:
plt.figure(figsize = (20,10))
plt.plot(data_raw['Close'][0:100])
plt.plot(data_raw['bollinger_hband'][0:100])
plt.plot(data_raw['bollinger_lband'][0:100])
plt.legend(['close', 'hband', 'lband'][0:100])
plt.show()
plt.figure(figsize = (20,10))
plt.plot(data_raw['obv'])
plt.show()

In [ ]:
data_raw['diff']  = data_raw['obv'].diff(periods=10)

In [ ]:
list(data_raw['obv'][0:100].diff(periods=10))

In [69]:
# creating new strategy
class bollinger_band:
    def __init__(self, ticker):
        self.ticker = ticker
        self.MAX_PROFIT_LIMIT = 0.2
        self.MAX_LOSS_LIMIT = -0.1
        self.current_position = 0
        self.output = 0
        self.positions = []
        self.logs = []
        # special - signal 
        self.signal = 0 # this signal changes once stock reaches the upper - bollinger band
    # define function
    def download_data(self):
        data_raw = yf.download(self.ticker+".NS", period="2y", interval = "1h")
        # calculating bollinger band
        volatile = ta.volatility.BollingerBands(data_raw['Close']) # type: ignore
        data_raw['bollinger_hband'] = volatile.bollinger_hband()
        data_raw['bollinger_lband'] = volatile.bollinger_lband()
        data_raw['bollinger_mavg'] = volatile.bollinger_mavg()
        data_raw['bollinger_pband'] = volatile.bollinger_pband()
        # calculating obv
        volatile_obv = ta.volume.OnBalanceVolumeIndicator(data_raw['Close'], data_raw['Volume']) #type: ignore
        data_raw['obv'] = volatile_obv.on_balance_volume()
        # calculating rsi
        data_raw['rsi'] = ta.momentum.RSIIndicator(data_raw['Close']).rsi() #type: ignore
        data_raw.dropna(inplace = True)
        data_raw['diff'] = data_raw['obv'].diff(periods=10) # slope

        self.data_list = data_raw.to_numpy()
        return data_raw
    # main function
    def run_test(self):
        self.download_data()
        for i in range(len(self.data_list)):
            # --
            try:
                current_node = self.data_list[i]
                # checking the bollinger signal 
                if(self.signal == 0):
                    if(current_node[6] <= current_node[3]):
                        self.signal = 1
                # buying section
                if(self.current_position <= 2 and self.signal == 1):
                    # checking [RSI] and [OBV - diff] condition
                    if(current_node[12] > 0 and current_node[11] > 50):

                        self.current_position += 1
                        self.positions.append(current_node[3])
                        self.signal = 0
                # selling section

                # selling with profit condition
                if(len(self.positions) != 0):
                    max_position = max(self.positions)
                    if((current_node[3] - max_position) / current_node[3]  >= self.MAX_PROFIT_LIMIT):
                        net_output = current_node[3] - max_position
                        print("selling the stock at {} with profit {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                        self.logs.append([current_node[0],current_node[3], net_output, "profit"])
                # selling with loss condititon

                    if((current_node[3] - max_position) / current_node[3]  < self.MAX_LOSS_LIMIT):
                        net_output = current_node[3] - max_position
                        print("selling the stock at {} with loss {}".format(current_node[3], net_output))
                        self.positions.pop(self.positions.index(max_position))
                        self.current_position -= 1
                        self.output += net_output
                        #self.logs.append([current_node[0],current_node[4], net_output, "loss"])
            except:
                pass
    
    def get_output(self):
        return self.output


            


